# Homework 2: SQL in Python (Data and Databases 2020)

The next four homework assignments will be in this iPython notebook format. Read the instructions as you go, and type in the necessary code--then run the cells to see if you got it right. Don't worry if you are new to python for this assignment--the only "code" you will be writing are SQL statements that you will paste into each myquery variable.


## The Challenges

For this homework we will be using the tables we imported from the [UN data site](http://data.un.org/Explorer.aspx). You should already have `wind` and `solar` installed. As you will see, this homework asks that you find one more table of your choice from the UN's Energy statistics an import it into the the same database. [I have posted the instructions here](http://floatingmedia.com/columbia/csv_to_sql.html)

These are relatively simple CSV files--the only really important columns are `country`, `year` and `usage`. But the tricky part is that most countries have multiple entries: one for every year the UN has been gathering data on that country. So most of these queries are going to ask you to get electrical usage results based on country and year. (Major hint: JOINs are going to need to be based on country AND year.)

In the questions below, we focus on exploratory queries--with an emphasis on aggregates and joins. Read the questions carefully, and then write SQL statements that will get you the correct results. (You will want to explore tables and test out queries directly in the shell before you paste them into Python--the error handling for SQL in Python can be confusing, and there should be no errors in the prewritten Python parts of this document. Also, sometimes an error in Python will break your connection with the database--see conn.rollback() below for that solution.


First things first: Connect to database! 
(Make sure the cell is selected, then go to the cell menubar and choose "Run Cells")

In [1]:
import pg8000
#Make sure to get the names of the database and user correct
#and to add whatever other parameters you need to connect your psql set up
conn = pg8000.connect(database="undata",user="sheridanwall")
#pg8000.connect(user, host='localhost', unix_sock=None, port=5432, database=None, password=None, ssl=None, timeout=None, application_name=None, tcp_keepalive=True)

#conn = pg8000.connect(database="undata")
print(type(conn))

<class 'pg8000.core.Connection'>


If the connection isn't working make sure that you have typed in the name of the database correctly, and your username as well. 

If you get an error when running one of the SQL cells below, run this conn.rollback() cell to restart your connection.

In [2]:
conn.rollback()

## Question 1: Simple(ish) Query 

Create a statement that searches for the top 15 countries for solar energy usage in 2014. So figure out first what table you are searching and what columns you need (you can figure out the columns by looking at the sample results below). Then you need to test by year so you only get results from 2014, and order it correctly.

```
Germany 36056.0 mil
United States 25764.0 mil
Japan 22952.0 mil
Italy 22306.0 mil
China 15189.0 mil
Spain 13673.0 mil
France 6391.0 mil
India 5020.0 mil
United Kingdom 4054.0 mil
Australia 4010.0 mil
Greece 3792.0 mil
Belgium 2883.0 mil
Korea, Republic of 2557.0 mil
Czechia 2123.0 mil
Canada 2120.0 mil
```

In [3]:
cursor = conn.cursor()
myquery = 'SELECT country, usage, year FROM solar WHERE year = 2014 ORDER BY usage DESC NULLS LAST LIMIT 15;'
cursor.execute(myquery)
for row in cursor:
    print(row[0], row[1], "mil")

Germany 36056.0 mil
United States 25764.0 mil
Japan 22952.0 mil
Italy 22306.0 mil
China 15189.0 mil
Spain 13673.0 mil
France 6391.0 mil
India 5020.0 mil
United Kingdom 4054.0 mil
Australia 4010.0 mil
Greece 3792.0 mil
Belgium 2883.0 mil
Korea, Republic of 2557.0 mil
Czechia 2123.0 mil
Canada 2120.0 mil


## Question 2: aggregate query

Using avg(), get the average solar energy usage for each country, and it display only the countries with an average that is above 500 (million). (Remember for aggregates you need to use HAVING to test values, instead of WHERE). Make sure it is properly ordered like the results below, and wrap your avg column in round() to get round numbers:

```
43567.0 China
10537.0 India
9627.0 Germany
8566.0 United States
7185.0 Japan
5217.0 Italy
3815.0 Spain
1961.0 United Kingdom
1886.0 France
1556.0 Greece
1361.0 Belgium
1316.0 Czechia
1275.0 Australia
1217.0 Thailand
1186.0 Chile
1036.0 Turkey
1002.0 South Africa
862.0 Korea, Republic of
858.0 Bulgaria
741.0 Honduras
700.0 Romania
684.0 Israel
633.0 Brazil
593.0 Canada
525.0 Pakistan
```

In [4]:
cursor = conn.cursor()
myquery = "SELECT round(AVG(usage)), country FROM solar GROUP BY country HAVING round(AVG(usage)) > 500 ORDER BY round(AVG(usage)) DESC NULLS LAST;"
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1])

43567.0 China
10537.0 India
9627.0 Germany
8566.0 United States
7185.0 Japan
5217.0 Italy
3815.0 Spain
1961.0 United Kingdom
1886.0 France
1556.0 Greece
1361.0 Belgium
1316.0 Czechia
1275.0 Australia
1217.0 Thailand
1186.0 Chile
1036.0 Turkey
1002.0 South Africa
862.0 Korea, Republic of
858.0 Bulgaria
741.0 Honduras
700.0 Romania
684.0 Israel
633.0 Brazil
593.0 Canada
525.0 Pakistan


## Question 3: aggregate query 2

Get a list of countries whose Solar Energy data first started being collected in 2012. This is tricky to figure out, but the query should be pretty simple. The main thing you need to figure out is which aggregate function will find the earliest date, and then you need to test if year is 2012. And make it alphabetical by country.


```
2012 American Samoa
2012 Botswana
2012 Gabon
2012 Lao People's Dem. Rep.
2012 Lithuania
2012 Marshall Islands
2012 Mauritius
2012 Nigeria
2012 Poland
2012 South Sudan
2012 St. Helena and Depend.
2012 St. Kitts-Nevis
2012 United Arab Emirates
2012 Venezuela (Bolivar. Rep.)
```


In [5]:
cursor = conn.cursor()
myquery = "SELECT MIN(year), country FROM solar GROUP BY country HAVING MIN(year)=2012 ORDER BY country;"
cursor.execute(myquery)
for row in cursor:
    print(row[0], row[1])

2012 American Samoa
2012 Botswana
2012 Gabon
2012 Lao People's Dem. Rep.
2012 Lithuania
2012 Marshall Islands
2012 Mauritius
2012 Nigeria
2012 Poland
2012 South Sudan
2012 St. Helena and Depend.
2012 St. Kitts-Nevis
2012 United Arab Emirates
2012 Venezuela (Bolivar. Rep.)


## Question 4: JOIN

Get the usage of solar AND wind power for Germany for each year in the tables. There should be four columns total, and you will be joining on two criteria.

(Please note that in the results below I am using Python to add in the strings "solar:" and "wind:")



```
Germany 1991 	wind: 215.0 	solar: 1.0
Germany 1992 	wind: 291.0 	solar: 4.0
Germany 1993 	wind: 674.0 	solar: 3.0
Germany 1994 	wind: 1428.0 	solar: 7.0
Germany 1995 	wind: 1712.0 	solar: 7.0
Germany 1996 	wind: 2078.0 	solar: 12.0
Germany 1997 	wind: 3034.0 	solar: 18.0
Germany 1998 	wind: 4593.0 	solar: 35.0
Germany 1999 	wind: 5528.0 	solar: 30.0
Germany 2000 	wind: 9352.0 	solar: 60.0
Germany 2001 	wind: 10456.0 	solar: 116.0
Germany 2002 	wind: 15856.0 	solar: 188.0
Germany 2003 	wind: 18713.0 	solar: 313.0
Germany 2004 	wind: 25509.0 	solar: 557.0
Germany 2005 	wind: 27229.0 	solar: 1282.0
Germany 2006 	wind: 30710.0 	solar: 2220.0
Germany 2007 	wind: 39713.0 	solar: 3075.0
Germany 2008 	wind: 40574.0 	solar: 4420.0
Germany 2009 	wind: 38647.0 	solar: 6584.0
Germany 2010 	wind: 37793.0 	solar: 11729.0
Germany 2011 	wind: 48883.0 	solar: 19599.0
Germany 2012 	wind: 50670.0 	solar: 26380.0
Germany 2013 	wind: 51708.0 	solar: 31010.0
Germany 2014 	wind: 57357.0 	solar: 36056.0
Germany 2015 	wind: 79206.0 	solar: 38726.0
Germany 2016 	wind: 78598.0 	solar: 38098.0
Germany 2017 	wind: 105693.0 	solar: 39401.0
```

Hint: after the join you need to test that the country is Germany, then order by what?

In [6]:
cursor = conn.cursor()
myquery = "SELECT solar.country, solar.year, windwind.usage, solar.usage FROM solar JOIN windwind ON solar.country = windwind.country AND solar.year = windwind.year WHERE solar.country = 'Germany' ORDER BY solar.year;"
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"\twind:",row[2],"\tsolar:",row[3])

Germany 1991 	wind: 215.0 	solar: 1.0
Germany 1992 	wind: 291.0 	solar: 4.0
Germany 1993 	wind: 674.0 	solar: 3.0
Germany 1994 	wind: 1428.0 	solar: 7.0
Germany 1995 	wind: 1712.0 	solar: 7.0
Germany 1996 	wind: 2078.0 	solar: 12.0
Germany 1997 	wind: 3034.0 	solar: 18.0
Germany 1998 	wind: 4593.0 	solar: 35.0
Germany 1999 	wind: 5528.0 	solar: 30.0
Germany 2000 	wind: 9352.0 	solar: 60.0
Germany 2001 	wind: 10456.0 	solar: 116.0
Germany 2002 	wind: 15856.0 	solar: 188.0
Germany 2003 	wind: 18713.0 	solar: 313.0
Germany 2004 	wind: 25509.0 	solar: 557.0
Germany 2005 	wind: 27229.0 	solar: 1282.0
Germany 2006 	wind: 30710.0 	solar: 2220.0
Germany 2007 	wind: 39713.0 	solar: 3075.0
Germany 2008 	wind: 40574.0 	solar: 4420.0
Germany 2009 	wind: 38647.0 	solar: 6584.0
Germany 2010 	wind: 37793.0 	solar: 11729.0
Germany 2011 	wind: 48883.0 	solar: 19599.0
Germany 2012 	wind: 50670.0 	solar: 26380.0
Germany 2013 	wind: 51708.0 	solar: 31010.0
Germany 2014 	wind: 57357.0 	solar: 36056.0
Germa

## Question 5: join with aggregates

Get the average wind usage and solar usage for each country. Order your results by average solar usage, and show only the top ten countries.

```
China wind: 139218.44 	solar: 43567.20
India wind: 11177.18 	solar: 10536.60
Germany wind: 29119.26 	solar: 9627.07
United States wind: 60074.25 	solar: 8566.00
Japan wind: 2079.79 	solar: 7185.18
Italy wind: 5026.96 	solar: 5217.21
Spain wind: 21045.68 	solar: 3814.57
United Kingdom wind: 9752.86 	solar: 1960.79
France wind: 6411.80 	solar: 1885.79
Greece wind: 1679.11 	solar: 1555.86
```

In [7]:
cursor = conn.cursor()
myquery = "SELECT solar.country, AVG(windwind.usage), AVG(solar.usage) FROM solar JOIN windwind ON solar.country = windwind.country GROUP BY solar.country ORDER BY AVG(solar.usage) DESC NULLS LAST LIMIT 10;"
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

China wind: 139218.44 	solar: 43567.20
India wind: 11177.18 	solar: 10536.60
Germany wind: 29119.26 	solar: 9627.07
United States wind: 60074.25 	solar: 8566.00
Japan wind: 2079.79 	solar: 7185.18
Italy wind: 5026.96 	solar: 5217.21
Spain wind: 21045.68 	solar: 3814.57
United Kingdom wind: 9752.86 	solar: 1960.79
France wind: 6411.80 	solar: 1885.79
Greece wind: 1679.11 	solar: 1555.86


## Question 6: same thing but slightly more complicated

Get the average wind usage and solar usage for each country, FROM 2010 TO THE PRESENT. Order your results by average solar usage, and show only the top ten countries.

```
China wind: 153258.25 	solar: 43567.20
Germany wind: 63738.50 	solar: 30124.88
United States wind: 173905.12 	solar: 27360.88
Japan wind: 5271.50 	solar: 23307.25
Italy wind: 14092.38 	solar: 18110.38
Spain wind: 48959.62 	solar: 12152.62
India wind: 29394.25 	solar: 10536.60
France wind: 17245.25 	solar: 5617.88
United Kingdom wind: 29249.12 	solar: 4646.38
Australia wind: 9072.75 	solar: 3861.88
```

In [17]:
cursor = conn.cursor()
myquery = "SELECT solar.country, AVG(windwind.usage), AVG(solar.usage) FROM solar JOIN windwind ON solar.country = windwind.country AND solar.year = windwind.year WHERE solar.year>=2010 AND windwind.year>=2010 GROUP BY solar.country ORDER BY AVG(solar.usage) DESC NULLS LAST LIMIT 10;"
cursor.execute(myquery)
for row in cursor:
    print(row[0],"wind:","%.2f" % row[1],"\tsolar:","%.2f" % row[2])

China wind: 203027.00 	solar: 43567.20
Germany wind: 63738.50 	solar: 30124.88
United States wind: 173905.12 	solar: 27360.88
Japan wind: 5271.50 	solar: 23307.25
Italy wind: 14092.38 	solar: 18110.38
Spain wind: 48959.62 	solar: 12152.62
India wind: 32940.20 	solar: 10536.60
France wind: 17245.25 	solar: 5617.88
United Kingdom wind: 29249.12 	solar: 4646.38
Australia wind: 9072.75 	solar: 3861.88


## Question 7: Building a basic timeseries

Step one: get a list of the first year of data and the most recent year of data for each country from the solar table. Limit the results to the first 15 with the oldest year and then in alphabetical order.

```
Italy 1990 -- 2017
Japan 1990 -- 2017
Korea, Republic of 1990 -- 2017
Mexico 1990 -- 2017
Portugal 1990 -- 2017
Spain 1990 -- 2017
Switzerland 1990 -- 2017
United States 1990 -- 2017
Finland 1991 -- 2017
Germany 1991 -- 2017
Netherlands 1991 -- 2017
Canada 1992 -- 2017
Australia 1993 -- 2017
Austria 1993 -- 2017
Liechtenstein 1993 -- 2018
```

In [9]:
cursor = conn.cursor()
myquery = 'SELECT country, MIN(year), MAX(year) FROM solar GROUP BY country ORDER BY MIN(year), country LIMIT 15;'
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"--",row[2])

Italy 1990 -- 2017
Japan 1990 -- 2017
Korea, Republic of 1990 -- 2017
Mexico 1990 -- 2017
Portugal 1990 -- 2017
Spain 1990 -- 2017
Switzerland 1990 -- 2017
United States 1990 -- 2017
Finland 1991 -- 2017
Germany 1991 -- 2017
Netherlands 1991 -- 2017
Canada 1992 -- 2017
Australia 1993 -- 2017
Austria 1993 -- 2017
Liechtenstein 1993 -- 2018


## Question 8: Building a timeseries (Bonus Question, skip if you need to--but don't skip the next one!)

Step two: Take that previous query, (order, limit and everything) and turn that into a subquery, and join the solar table on it twice, first to get the usage from the earliest date, next get the usage from the most recent date. Order it by the most usage from the most recent year.

Note: to join at the same table twice, you need to alias the table (solar as sl1, solar as sl2). I have given you the template for a subquery below. This is getting very challenging, so how far you can get. 
```
United States 1990 -- 2017 usage change:  666.0 -- 70980.0
Japan 1990 -- 2017 usage change:  67.0 -- 55068.0
Germany 1991 -- 2017 usage change:  1.0 -- 39401.0
Italy 1990 -- 2017 usage change:  4.0 -- 24378.0
Spain 1990 -- 2017 usage change:  9.0 -- 14397.0
Australia 1993 -- 2017 usage change:  11.0 -- 8073.0
Korea, Republic of 1990 -- 2017 usage change:  1.0 -- 7057.0
Canada 1992 -- 2017 usage change:  2.0 -- 3573.0
Netherlands 1991 -- 2017 usage change:  1.0 -- 2204.0
Switzerland 1990 -- 2017 usage change:  1.0 -- 1683.0
Austria 1993 -- 2017 usage change:  1.0 -- 1269.0
Mexico 1990 -- 2017 usage change:  1.0 -- 1085.0
Portugal 1990 -- 2017 usage change:  1.0 -- 992.0
Finland 1991 -- 2017 usage change:  1.0 -- 43.0
Liechtenstein 1993 -- 2018 usage change:  0.008 -- 24.4
```

In [10]:
cursor = conn.cursor()
myquery = '''
SELECT
FROM (put your entire query from question #7 in here) as tseries
JOIN solar as sl1 ...
JOIN solar as sl2 ...
ORDER BY
'''  
cursor.execute(myquery)
for row in cursor:
    print(row[0],row[1],"--",row[2],"usage change: ", row[3],"--",row[4])

ProgrammingError: {'S': 'ERROR', 'V': 'ERROR', 'C': '42601', 'M': 'syntax error at or near "entire"', 'P': '24', 'F': 'scan.l', 'L': '1180', 'R': 'scanner_yyerror'}

## Question 9: join with aggregates, new data

Your final task: go back to the [UN data site](http://data.un.org/Explorer.aspx) and download another table of your choice from the energy statistics. [Full instructions here.](http://floatingmedia.com/columbia/csv_to_sql.html) Choose a table that has similar columns to the wind and energy tables, and input it into your database.  Make sure you test that everything is working in the shell before going back to Python.

Now, write a query that joins the third table to the query you made in Question 5. Get the average use for each Energy type: Solar, Wind, and the table you chose. Order it by the energy type you chose (that column should come first after the country). And display the top 10 countries by the average of the energy type that you added, along with the averages for solar and wind.


In [19]:
cursor = conn.cursor()
myquery = "SELECT solar.country, AVG(windwind.usage) AS wind, AVG(solar.usage) AS solar, AVG(hydro.usage) AS hydro FROM solar JOIN windwind ON solar.country = windwind.country JOIN hydro ON solar.country = hydro.country GROUP BY solar.country ORDER BY hydro DESC NULLS LAST LIMIT 10;"
cursor.execute(myquery)
for row in cursor:
    print(row[0],"%.2f" % row[1],"%.2f" % row[2],"%.2f" % row[3])

China 139218.44 43567.20 479493.00
Canada 6817.58 592.65 353221.79
Brazil 14780.83 632.83 321711.00
United States 60074.25 8566.00 300299.14
Russian Federation 28.95 378.75 171180.50
India 11177.18 10536.60 94655.11
Japan 2079.79 7185.18 92439.36
Sweden 3551.64 25.00 67724.68
France 6411.80 1885.79 66598.00
Venezuela (Bolivar. Rep.) 94.27 4.87 65899.00


Great work! You're done.